##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Базовая классификация текста

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве демонстрируется классификация текста, начиная с простых текстовых файлов, хранящихся на диске. Вы научите бинарный классификатор выполнять анализ тональности набора данных IMDB. В конце руководства есть упражнение, которое вы можете попробовать научить мультиклассовый классификатор предсказывать тег для вопроса программирования на Stack Overflow.

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
print(tf.__version__)

## Анализ настроений

Это руководство обучает модель анализа настроений классифицировать обзоры фильмов как *положительные* или *отрицательные* на основе текста. Это пример *бинарной* - или двухклассовой классификации, важного и широко применяемого вида машинного обучения.

Вы будете использовать [Большой набор данных обзоров фильмов](https://ai.stanford.edu/~amaas/data/sentiment/), который содержит текст 50 000 обзоров фильмов из [База данных фильмов в Интернете](https://www.imdb.com/). Они разделены на 25 000 отзывов для обучения и 25 000 отзывов для тестирования. Наборы для обучения и тестирования *сбалансированы*, что означает, что они содержат равное количество положительных и отрицательных отзывов.


### Загрузка и изучение набора данных IMDB

Давайте загрузим и извлечем набор данных, а затем исследуем структуру каталогов.

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [ ]:
os.listdir(dataset_dir)

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

Каталоги `aclImdb/train/pos` и `aclImdb/train/neg` содержат множество текстовых файлов, каждый из которых представляет собой отдельный обзор фильма. Давайте посмотрим на один из них.

In [ ]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

### Загрузка датасета

Теперь вы загрузите данные с диска и подготовите их в формате, подходящем для обучения. Для этого вы будете использовать полезную утилиту [text_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text_dataset_from_directory), которая ожидает следующую структуру каталогов.

```
main_directory/
...class_a/
......a_text_1.txt
......a_text_2.txt
...class_b/
......b_text_1.txt
......b_text_2.txt
```

Чтобы подготовить набор данных для бинарной классификации, вам потребуются две папки на диске, `class_a` и` class_b`. Это будут положительные и отрицательные обзоры фильмов, которые можно найти в  `aclImdb/train/pos` и `aclImdb/train/neg`. Поскольку набор данных IMDB содержит дополнительные папки, вы удалите их перед использованием этой утилиты.

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Затем вы воспользуетесь утилитой `text_dataset_from_directory`, чтобы создать размеченный `tf.data.Dataset`. [tf.data](https://www.tensorflow.org/guide/data) - мощный набор инструментов для работы с данными.

При проведении эксперимента с машинным обучением рекомендуется разделить набор данных на три части: [train](https://developers.google.com/machine-learning/glossary#training_set), [validation](https://developers.google.com/machine-learning/glossary#validation_set) и [test](https://developers.google.com/machine-learning/glossary#test-set).

Набор данных IMDB уже разделен на обучающий и тестовый, но в нем отсутствует набор для проверки. Давайте создадим набор для проверки, используя разделение обучающих данных 80:20 с помощью аргумент `validation_split`.

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Как вы видели выше, в папке для обучения 25 000 примеров, 80%(или 20 000) которых вы будете использовать  для обучения. Вы можете обучить модель, передав набор данных непосредственно в `model.fit`. Если вы новичок в `tf.data`, ниже показан пример того, как можно получить и просмотреть несколько примеров из готового датасета.

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Обратите внимание, что обзоры содержат необработанный текст(с пунктуацией и случайными HTML-тегами, такими как `<br/>`). В следующем разделе вы узнаете, как с ними справиться.

Метки равны 0 или 1. Чтобы увидеть, какие из них соответствуют положительным и отрицательным обзорам, вы можете проверить свойство `class_names` в наборе данных.

In [ ]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Далее вы создадите набор данных для проверки и тестирования. Вы будете использовать оставшиеся 5000 отзывов из обучающего набора для проверки.

Примечание. При использовании аргументов `validation_split` и `subset` убедитесь, что вы указали случайное начальное число(random seed) или передали `shuffle=False`, чтобы проверочные и обучающие пакеты не перекрывались.

In [ ]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

In [ ]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Примечание. API предварительной обработки, используемые в следующем разделе, являются экспериментальными в TensorFlow 2.3 и могут быть изменены.

### Подготовка набор данных для обучения

Теперь вы стандартизируете, токенизируете и векторизуете данные, используя полезный слой `preprocessing.TextVectorization`.

Стандартизация относится к предварительной обработке текста, обычно она используется для удаления знаков препинания или элементов HTML. Токенизация относится к разделению строк на токены(например, разделение предложения на отдельные слова путем разделения на пробелы). Векторизация относится к преобразованию токенов в числа, чтобы их можно было передать в нейронную сеть. Все эти задачи могут быть выполнены с помощью этого слоя.

Как вы видели выше, обзоры содержат различные HTML-теги, такие как `<br />`. Эти теги не будут удалены стандартизатором в слое `TextVectorization`, так как по умолчанию `TextVectorization` преобразует текст в нижний регистр и удаляет знаки пунктуации, но не удаляет HTML. Вы напишете специальную функцию стандартизации для удаления HTML.

Примечание. Чтобы предотвратить [перекос при обучении/тестировании](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew)(также известный как перекос при обучении/обслуживании), важно сделать идентичную предварительную обработку данных как для обучения, так и для обучения и тестирования. Чтобы упростить этот процесс, слой `TextVectorization` можно включить непосредственно в вашу модель, как показано далее в этом руководстве.

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

Затем вы создадите слой `TextVectorization`. Вы будете использовать этот слой для стандартизации, токенизации и векторизации данных. Укажите значение `int` для аргумента `output_mode` , чтобы создать уникальные целочисленные индексы для каждого токена.

Обратите внимание, что вы используете функцию разделения по умолчанию и пользовательскую функцию стандартизации, которую вы определили выше. Вы также определите некоторые константы для модели, например, значение `sequence_length`, которое заставит слой дополнять или обрезать последовательности до значения `sequence_length`.

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

Затем вы вызовете `adapt`, чтобы подогнать состояние уровня предварительной обработки к набору данных. Это заставит модель построить индекс строки -> целые числа.

Примечание: при вызове `adapt` важно использовать только тренировочный датасет, использование тестового датасета приведет к утечке информации.

In [ ]:
# Создайте набор данных только из текста(без меток), затем вызовите adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

Давайте создадим функцию, чтобы увидеть результат использования слоя предварительной обработки для некоторых данных.

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
# получить пакет(из 32 отзывов и меток) из набора данных
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Как вы можете видеть выше, каждый токен был заменен целым числом. Вы можете найти токен(строку), которому соответствует каждое целое число, вызвав метод уровня `.get_vocabulary()`.

In [ ]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

Вы почти готовы обучать свою модель. В качестве последнего шага предварительной обработки вы примените слой `TextVectorization`, ко всем датасетам - для обучения, проверки и тестирования.

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### Настройка датасета для повышения производительности

Существует два важных метода, которые вы должны использовать при загрузке данных, чтобы убедиться, что ввод-вывод не становится блокирующим.

`.cache()` сохраняет данные в памяти после их загрузки с диска. Этот метода гарантирует, что набор данных не станет узким местом при обучении вашей модели. Если ваш набор данных слишком велик для размещения в памяти, вы также можете использовать этот метод для создания производительного кеша на диске, который более эффективен для чтения, чем многие небольшие файлы.

`.prefetch()` перекрывает предварительную обработку данных и выполнение модели во время обучения.

Вы можете узнать больше об обоих методах, а также о том, как кэшировать данные на диск, в [руководстве по производительности данных](https://www.tensorflow.org/guide/data_performance).

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Создание модели

In [ ]:
embedding_dim = 16

In [ ]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Для построения классификатора мы используем стек слоев:

1. Первый слой - это `Embedding`. Этот слой принимает обзоры, закодированные в целочисленном формате, и ищет вектор внедрения для каждого индекса слова. Эти векторы изучаются по мере обучения модели. Векторы добавляют измерение к выходному массиву. В результате получаются следующая размерность: `(batch, sequence, embedding)`. Чтобы узнать больше о технике эмбеддинга, см. [Руководство по эмбеддингу слов](../text/word_embeddings.ipynb).
2. Затем слой `GlobalAveragePooling1D`, который возвращает выходной вектор фиксированной длины для каждого примера путем усреднения по размерности. Это позволяет модели обрабатывать ввод переменной длины самым простым из возможных способов.
3. Этот выходной вектор фиксированной длины передается по конвейеру через полносвязанный слой(`Dense`) с 16 скрытыми узлами.
4. Последний слой - полносвязанный слой(`Dense`) с единственным выходным узлом.

### Функция потерь и оптимайзер

Для обучения модели нужны функция потерь и оптимайзер. Поскольку это проблема бинарной классификации и модель выводит вероятность(единичный слой с активацией сигмоид), вы будете использовать функцию потерь `losses.BinaryCrossentropy`.

Теперь настройте модель для использования оптимайзера и функции потерь:

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

### Обучение модели

Вы обучите модель, передав объект `dataset` методу `fit`.

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

### Оценка модели

Посмотрим, как модель работает. Будут возвращены два значения. Величина потерь(число, которое представляет нашу ошибку, меньшие значения лучше) и точность.


In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Этот довольно наивный подход обеспечивает точность около 86%.

### Создание графика точности и потерь

`model.fit()` возвращает объект `History`, который содержит словарь со всем, что произошло во время обучения:

In [ ]:
history_dict = history.history
history_dict.keys()

Есть четыре записи: по одной для каждой отслеживаемой метрики во время обучения и проверки. Вы можете использовать их для построения графика потерь при обучении и проверке для сравнения, а также для определения точности обучения и проверки:

In [ ]:
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

Точки на графике представляют потери и точность при обучении, а сплошные линии - потери и точность при оценке.

Обратите внимание, что потери при обучении *уменьшаются* с каждой эпохой, а точность обучения *увеличивается* с каждой эпохой. Это ожидаемое поведение при использовании оптимизации градиентного спуска - он должен минимизировать ошибку на каждой итерации.

Это не относится к потерям и точности при оценке - они достигают пика раньше точности обучения. Это пример переобучения: модель лучше работает с данными обучения, чем с данными, которых она никогда раньше не видела. После определенной точки модель чрезмерно оптимизирует и изучает признаки, *специфичные* для обучающих данных, которые не *характерны* для тестовых данных.

В этом конкретном случае вы можете избежать переобучения, просто остановив тренировку модели, когда точность проверки больше не увеличивается. Один из способов сделать это - использовать обратный вызов [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping?version=nightly).

## Экспорт модели

В приведенном выше коде вы применили слой `TextVectorization` к набору данных перед отправкой текста в модель. Если вы хотите, чтобы ваша модель могла работать с необработанными строками(например, чтобы упростить ее развертывание), вы можете включить слой `TextVectorization` в вашу модель. Для этого вы можете создать новую модель, используя только что обученные веса.

In [ ]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Протестируйте ее с помощью вызова raw_test_ds, который возвращает необработанные строки
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

### Вывод для новых данных

Чтобы получить прогнозы для новых данных, вы можете просто вызвать `model.predict()`.

In [ ]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

export_model.predict(examples)

Включение логики предварительной обработки текста в модель позволяет экспортировать модель в продакшен, что упрощает развертывание и снижает вероятность [перекоса обучения/тестирования](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serve_skew).

При выборе места для применения слоя `TextVectorization` следует учитывать разницу в производительности. Использование его вне модели позволяет выполнять асинхронную обработку на CPU и буферизацию данных при обучении на GPU. Таким образом, если вы тренируете свою модель на графическом процессоре, вы, вероятно, захотите использовать этот вариант, чтобы получить максимальную производительность при разработке модели, а затем переключитесь на включение слоя `TextVectorization` в вашу модель, когда вы будете готовы к развертыванию.

Посмотрите этот [учебник](https://www.tensorflow.org/tutorials/keras/save_and_load), чтобы узнать больше о сохранении моделей.

## Упражнение: мультиклассовая классификация по вопросам со Stack Overflow

В этом руководстве показано, как обучить бинарный классификатор на наборе данных IMDB. В качестве упражнения вы можете изменить этот ноутбук, чтобы обучить мультиклассовый классификатор предсказывать тег вопроса по программированию на [Stack Overflow](http://stackoverflow.com/).

Мы подготовили для вас [набор данных](http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz), содержащий несколько тысяч вопросов по программированию(например, «Как можно отсортировать словарь по значению в Python?»), размещенных на Stack Overflow. Каждый из них помечен ровно одним тегом(Python, CSharp, JavaScript или Java). Ваша задача - взять вопрос в качестве входных данных и предсказать соответствующий тег, в данном случае Python.

Набор данных, с которым вы будете работать, содержит несколько тысяч вопросов, извлеченных из гораздо большего общедоступного набора данных Stack Overflow на [BigQuery](https://console.cloud.google.com/marketplace/details/stack-exchange/stack-overflow), который содержит более 17 миллионов сообщений.

После загрузки датасета вы обнаружите, что он имеет структуру каталогов, аналогичную датасету IMDB, с которым вы работали ранее:

```
train/
...python/
......0.txt
......1.txt
...javascript/
......0.txt
......1.txt
...csharp/
......0.txt
......1.txt
...java/
......0.txt
......1.txt
```

Примечание: чтобы усложнить задачу классификации, мы заменили любые слова Python, CSharp, JavaScript или Java в вопросах программирования словом *blank*(так как многие вопросы содержат название языка).

Чтобы выполнить это упражнение, вам следует изменить этот ноутбук для работы с набором данных Stack Overflow, внеся следующие изменения:

1. В верхней части записной книжки обновите код, загружающий набор данных IMDB, на код для загрузки [набора данных Stack Overflow](http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz). Поскольку набор данных Stack Overflow имеет аналогичную структуру каталогов, вам не нужно будет вносить много изменений.

1. Измените последний слой вашей модели так, чтобы он читался как `Dense(4)`, поскольку теперь существует четыре выходных класса.

1. При компиляции модели измените функцию потерь на [SparseCategoricalCrossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy?version=nightly). Это правильная функция потерь для использования в задаче классификации нескольких классов, когда метки для каждого класса являются целыми числами (в нашем случае они могут быть *0*, *1*, *2* или *3*).

1. После внесения этих изменений вы сможете обучать мультиклассовый классификатор.

Если же у вас возникли проблемы, вы можете найти готовое решение [здесь](https://github.com/tensorflow/examples/blob/master/community/en/text_classification_solution.ipynb).

## Узнать больше

В этом руководстве была описана классификация текста с нуля. Чтобы узнать больше о рабочем процессе классификации текста в целом, мы рекомендуем прочитать [это руководство](https://developers.google.com/machine-learning/guides/text-classification/) от разработчиков Google.